In [2]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 14.9 MB/s eta 0:00:00


In [4]:
pip install neptune-tensorflow-keras


In [6]:
pip install neptune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00
  Created wheel for bravado-core: filename=bravado_core-6.1.1-py2.py3-none-any.whl size=67675 sha256=c05fefe8f9837f2bbf5c223463a705dbfa8469c233c9ac9b6fece84fd3fbfadf
  Stored in directory: /root/.cache/pip/wheels/cf/d7/1c/1d707a21e0a0323bdbfbb2f6de125ae6bb70d62aa2838df321
Successfully built bravado-core


In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
from neptune_tensorflow_keras import NeptuneCallback

ModuleNotFoundError: No module named 'optuna'

In [8]:
import neptune

run = neptune.init_run(
    project="alesya3613/Credit",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTUwOWQ5MC0wNDA1LTQzZWUtYmVhZi05ZjhmOGJiNTdkMmUifQ==",
)  # your credentials


[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/alesya3613/Credit/e/CRED-3


In [4]:
df = pd.read_csv('/loan_data.csv')

In [ ]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [5]:
from sklearn.preprocessing import LabelEncoder
 # Создаем LabelEncoder
label_encoders = {}  # Словарь для хранения энкодеров

# Применяем LabelEncoder ко всем нечисловым столбцам
for col in df.select_dtypes(include=["object", "category"]).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Кодируем
    label_encoders[col] = le  # Сохраняем энкодер

In [6]:
y = df['loan_status'].values
X = df.drop(columns = ['loan_status']).values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [14]:
def objective(trial):

    params = {
        'learning_rate': [0.1, 0.01, 0.001],
        'batch_size': [32,50, 45],
        'epochs': [50, 100, 350, 500, 400,750],
        'layers': [35, 50, 75, 100, 250, 350, 500]
    }
    run['parameters'] = params

    learning_rate = trial.suggest_categorical("learning_rate", params['learning_rate'])
    batch_size = trial.suggest_categorical("batch_size", params['batch_size'])
    epochs = trial.suggest_categorical("epochs", params['epochs'])
    layers = trial.suggest_categorical("layers", params['layers'])

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])

    optimizer = {
        "Adam": tf.keras.optimizers.Adam(learning_rate),
        "SGD": tf.keras.optimizers.SGD(learning_rate),
        "RMSprop": tf.keras.optimizers.RMSprop(learning_rate),
    }[optimizer_name]

    # Создаём модель
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    cbk = NeptuneCallback(run=run, base_namespace='metrics')

    # Обучаем модель
    model.fit(X_train, y_train, epochs=5, batch_size=batch_size, verbose=0, callbacks=cbk)

    # Оцениваем модель
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Логируем параметры и результаты в Neptune
    run[f"trial/{trial.number}/learning_rate"] = float(learning_rate)  # Приводим к float
    run[f"trial/{trial.number}/batch_size"] = int(batch_size)
    run[f"trial/{trial.number}/epochs"] = int(epochs)
    run[f"trial/{trial.number}/layers"] = int(layers)
    run[f"trial/{trial.number}/optimizer"] = optimizer_name

    # Логируем финальную точность через .log()
    run[f"trial/{trial.number}/final_accuracy"].log(accuracy)
    return accuracy

In [15]:
study = optuna.create_study(direction="maximize")  # Максимизируем точность
study.optimize(objective, n_trials=20)

# Логируем лучший результат
run["best_params"] = str(study.best_params)
run["best_accuracy"] = float(study.best_value)

# Завершаем Neptune
run.stop()

[I 2025-03-23 12:17:23,850] A new study created in memory with name: no-name-a762c2fd-f847-477b-8263-c46697236b5f
[neptune] [warning] NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'list'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type


[I 2025-03-23 12:17:42,484] Trial 0 finished with value: 0.7766666412353516 and parameters: {'learning_rate': 0.1, 'batch_size': 32, 'epochs': 100, 'layers': 500, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.7766666412353516.


[I 2025-03-23 12:18:02,456] Trial 1 finished with value: 0.7766666412353516 and parameters: {'learning_rate': 0.001, 'batch_size': 45, 'epochs': 350, 'layers': 50, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.7766666412353516.


[I 2025-03-23 12:18:13,929] Trial 2 finished with value: 0.8193333148956299 and parameters: {'learning_rate': 0.001, 'batch_size': 50, 'epochs': 400, 'layers': 250, 'optimizer': 'RMSprop'}. Best is trial 2 with value: 0.8193333148956299.


[I 2025-03-23 12:18:26,172] Trial 3 finished with value: 0.8275555372238159 and parameters: {'learning_rate': 0.001, 'batch_size': 50, 'epochs': 750, 'layers': 50, 'optimizer': 'RMSprop'}. Best is trial 3 with value: 0.8275555372238159.


...

[I 2025-03-23 12:21:32,289] Trial 16 finished with value: 0.7766666412353516 and parameters: {'learning_rate': 0.1, 'batch_size': 45, 'epochs': 350, 'layers': 75, 'optimizer': 'RMSprop'}. Best is trial 3 with value: 0.8275555372238159.


[I 2025-03-23 12:21:44,534] Trial 17 finished with value: 0.22333332896232605 and parameters: {'learning_rate': 0.001, 'batch_size': 50, 'epochs': 750, 'layers': 35, 'optimizer': 'RMSprop'}. Best is trial 3 with value: 0.8275555372238159.


[I 2025-03-23 12:21:56,370] Trial 18 finished with value: 0.7766666412353516 and parameters: {'learning_rate': 0.001, 'batch_size': 50, 'epochs': 400, 'layers': 100, 'optimizer': 'SGD'}. Best is trial 3 with value: 0.8275555372238159.


[I 2025-03-23 12:22:08,573] Trial 19 finished with value: 0.7766666412353516 and parameters: {'learning_rate': 0.1, 'batch_size': 45, 'epochs': 750, 'layers': 50, 'optimizer': 'Adam'}. Best is trial 3 with value: 0.8275555372238159.


[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 14 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 14 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/alesya3613/Credit/e/CRED-3/metadata


In [11]:
pip install scikeras


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [21]:
# Определение модели
model = RandomForestClassifier()

# Словарь гиперпараметров
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2)

# Обучение
grid_search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)
print("Лучший результат:", grid_search.best_score_)

# Применение лучшей модели
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Оценка точности
print("Точность на тестовой выборке:", accuracy_score(y_test, y_pred))


Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.8s
[CV] END max_depth=N

Лучшие параметры: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

Лучший результат: 0.9256388888888889

Точность на тестовой выборке: 0.9276666666666666